# This is an attempt to learn by building and training an LLM from Scratch  

In [2]:
import os
import torch
from torch import Tensor
import urllib.request
    
# Check which GPU if any is available
# torch.cuda.is_available():
#     device = torch.device("cuda")
#     x: Tensor = torch.ones(1, device=device)
#     print(f"x = {x} using 'cuda:0' backend")
#     
# elif 
if torch.backends.mps.is_available():
    device = torch.device("mps")
    x: Tensor = torch.ones(1, device=device)
    print(f"x = {x} using {device} backend")
else:
    device = torch.device("cpu")
    # x: Tensor = torch.ones(1, device=device)
 
print(device)

def get_some_text():
    # Now lets load the text data
    bookUrl = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"  # Download a text (book)
    filepath = "../data/the-verdict.txt"
    # print(file_path)
    ra
    if not os.path.exists(filepath):
        urllib.request.urlretrieve(bookUrl, filepath)

    with open(filepath, "r", encoding="utf-8") as f:
        rawtext = f.read()
        
    print("Total characters in the story: ", len(rawtext))
    print("Total Lines in raw text: ", rawtext.count("\n"))
    return rawtext

raw_text = get_some_text()
print("Some text: ", raw_text[:49])

x = tensor([1.], device='mps:0') using mps backend
mps


FileNotFoundError: [Errno 2] No such file or directory: '../data/the-verdict.txt'

In [ ]:
# Now we have to tokenize the text. The best way to do that is to use a pre-build tokennizer, but first we will try some 
# basic python regular expressions to do the same things
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
#
print(len(preprocessed))
print(preprocessed[:30])

In [ ]:
# Now we need to generate token IDs
# Now let us create a list of all unique tokens and sort them alphabetically to determine the vocabulary size
all_uniq_words = sorted(set(preprocessed))
vocab_size = len(all_uniq_words)
print("Vocab size: ", vocab_size)

# Now that we know the vocabulary size, lets enumerate and assign some numbers to them
vocab = {token:integer for integer,token in enumerate(all_uniq_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break


In [ ]:
from src.chapter02.SimpleTokenizerV1 import SimpleTokenizerV1

# Now we want to apply this vocabulary to convert new text to generate token id
# When we want to convert the outputs of an LLM from numbers back into text, we need a way to turn token IDs into text. 
# For this, we can create an inverse version of the vocabulary that maps token IDs back to the corresponding text tokens.

tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted you know," 
        Mrs Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

In [ ]:
all_tokens = sorted(list(set(preprocessed))) # Make preprocessed a list so we can extend it
all_tokens.extend(["<|unk|>", "<|endoftext|>"])
# redo the vocab population
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))


In [ ]:
# Print the last 5 vocab items
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
from src.chapter02.SimpleTokenizerV2 import SimpleTokenizerV2

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

In [ ]:
### Byte Pair Encoding 
from importlib.metadata import version
import tiktoken
print("Tiktoken version: ", version("tiktoken"))
#print("Tiktoken version: ", tiktoken.__version__)

In [ ]:
#### This is the tokenizer using the GPT2 tokenization model
tokenizer = tiktoken.get_encoding("gpt2")
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(f"Encoded: {integers}")
strings = tokenizer.decode(integers)
print(f"Decoded: {strings}")

In [ ]:
print(tokenizer.encode("Akwirw ier"))
print(tokenizer.decode(tokenizer.encode("Akwirw ier")))

In [ ]:
# Let's now do Data Sampling with a sliding window
# 1. Let's tokenize the entire story with BPE tokenizer first

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))
enc_sample = encoded_text[50:]

# Now Let's start by defining x and y where x has input tokens and y the output tokens shifted by 1
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
# print(f"x: {x}")
# print(f"y:      {y}")


#####
# Next word prediction tasks can now be created by 
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    # print(f"context input: {context} --> desired prediction: {desired}")
    # Now we create the input output target pairs
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

In [ ]:
# from Dataloader import Dataloader
# 
# dataloader = Dataloader(batch_size=8, max_length=4, stride=4, shuffle=False, drop_last=True, num_workers=0)
# dataloader = dataloader.get_instance(file_path, text_enc='utf-8', mode='r')
# if dataloader is not None:
#     data_iter = iter(dataloader)
#     inputs, targets = next(data_iter)
#     print("Loaded text data...\n")
#     print("Inputs: \n", inputs)
#     print("\nTargets: \n", targets)
# else: 
#     print("Failed loading ", dataloader)


In [ ]:
import torch.nn
from src.chapter02.Dataloader import Dataloader
file_path = "../data/the-verdict.txt"

####
# Finally we need to create the embeddings for the tokens
# If we have a batch size of 8 with 4 tokens each it'll be an 8 x 4 x 256 tensor
max_length = 4  

mydataloader = Dataloader(batch_size=8, max_length=max_length, stride=4, shuffle=False, drop_last=True, num_workers=0)
dataloader = mydataloader.create_dataloader_v1(txt=raw_text)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
# print("Input Token IDs:\n", inputs)
# print("Input tensor shape: ", inputs.shape) 

# Now since self-attentions are position agnostic, we should add some positional data.
# Absolute and relative positional data can be added. So let's create embeddings with say 256 dimensions
vocab_size = 50257
output_dim = 256
# context_length = 1024

## Now lets embed the input tensors
token_embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=output_dim)
token_embeddings = token_embedding_layer(inputs)
print("Token embeddings shape: ", token_embeddings.shape) #8x4x256


# For a GPT model’s absolute position embedding approach, we just need to create another embedding 
# layer that has the same embedding dimension as the token_embedding_ layer:
context_length = max_length     #context is length of positions we care about for attention
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print("Positional Embeddings Shape: ", pos_embeddings.shape) # 4x256
#
# Add the positional embeddings to token embeddings
input_embeddings = token_embeddings + pos_embeddings
print("Position Merged Input Embeddings Shape: ", input_embeddings.shape)
#
# Now lets look at the dataloader
for batch in dataloader:
    inputs, targets = batch
    token_embeddings = token_embedding_layer(inputs)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))
    input_embeddings = token_embeddings + pos_embeddings
    break
#
print("Batch Embeddings Shape: ", input_embeddings.shape)
    
print("Input tensor ", x)
print("Target tensor", y)

In [ ]:
# Chapter 3 - Attention
#
import torch

# In self-attention our goal is to calculate context vector z(i) for each 
# element x(i) of the input sequence. Consider the following input sequence 
#
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
# inputs.to(device)
print("Input sequence shape: ", inputs.shape)
# 
# Now calculate weights for attention
# Assume query is the second word "journey" or inputs[1] 
#
query = inputs[1]
# query.to(device)
print(f"Query is the 2nd word 'journey': {query}")
#
attention_scores_2 = torch.empty(inputs.shape[0])
# attention_scores_2.to(device)
for idx, x_i in enumerate(inputs):
    attention_scores_2[idx] = torch.dot(x_i, query)
#    print(f"Sequence Element [{idx}], attention_score: {attention_scores_2}")
print(f"Final value of attention_score_2: {attention_scores_2}")


In [ ]:

## Note: For all elements if we were to calculate attention it'd be a O(n^2) operation
# NOW we normalize the attention weights, so they sum up to 1
attention_weights_2_tmp = attention_scores_2 / attention_scores_2.sum()
# attention_weights_2_tmp.to(device)
print("Normalized attention weights:", attention_weights_2_tmp)
print("Sum of attention weights:", attention_weights_2_tmp.sum())

## Generally we normalize using the softmax to do the normalization

# # define a softmax function
def softmax_naive(tensor_x):
    return torch.exp(tensor_x) / torch.exp(tensor_x).sum(dim=0, keepdim=True)
# 

attention_scores_2_naive = softmax_naive(attention_scores_2)
# attention_scores_2_naive.to(device)

print("Attention weights naive:", attention_scores_2_naive)
print ("Naive Sum: ", attention_scores_2_naive.sum())
# 
# Generally we normalize using the torch.softmax() to do the normalization
# Softmax ensures its always positive and always adds up to 1
#
attention_weights_2_torch_softmax = torch.softmax(attention_scores_2, dim=0)
# attention_weights_2_torch_softmax.to(device)
print("Attention weights torch softmax:", attention_weights_2_torch_softmax)
# print("Attention weights torch softmax Sum: ", attention_weights_2_torch_softmax.sum())

# Now that we have calculated the normalized attention weights, we are ready for the final step.
# Calculate the context vector z(2) by multiplying the embedded input tokens x(i), 
# with the corresponding normalized attention weights and then summing the resultant vectors
#
query = inputs[1]
# query.to(device)
#
context_vec_2 = torch.zeros(query.shape)
# context_vec_2.to(device)
#
for i, x_i in enumerate(inputs):
    context_vec_2 += (attention_weights_2_torch_softmax[i] * x_i)
print("Context vector z2: ", context_vec_2)

#
# Now in similar fashion lets calculate attention scores for all the input sequences 
attention_scores = torch.empty(inputs.shape[0],inputs.shape[0])
# attention_scores.to(device)
print("\nAttention Scores matrix shape: ", attention_scores.shape)
#
# Using for loops
#
# for i, x_i in enumerate(inputs):
#     for j, x_j in enumerate(inputs):
#         attention_scores[i, j] = torch.dot(x_i, x_j)
# #
#print(attention_scores)
#
# Using matrix multiplication we can do it faster
#
attention_scores_m = inputs @ inputs.T
# attention_scores_m.to(device)
#print("Normalized attention scores \n", attention_scores_m)

# Just as before lets normalize the rows, so they sum up to 1
# NOTE: Here dim = -1 means we are applying the softmax along the last dimension of the attention_scores_m tensor
#
attention_weights = torch.softmax(attention_scores_m, dim=-1)
# attention_weights.to(device)
#print("Normalized ATTENTION weights \n", attention_weights)
# print("Softmax Sums:\n", attention_weights.sum(dim=-1))

# FINAL STEP
# Now let's calculate the context vectors for all the input by multiplying the input with attention weights
all_context_vectors = attention_weights @ inputs # Matrix multiplication
# all_context_vectors.to(device)
#print("Context vector for the entire sequence\n", all_context_vectors)

In [ ]:

###
### 3.4.1 Using weighted matrix
###
#
# Computing the attention weights step by step
#
torch.manual_seed(123)
x_2 = inputs[1]
# x_2.to(device)
d_in = inputs.shape[1]
d_out = 2
print("x_2: ", x_2)
# Now let's initialize 3 weighted matrices Wq, Wk and Wv
# Setting requires_grad = False, to reduce clutter, but for model training this should be set to True

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
#
# Next we compute the query, key and value vectors
# Note the output is a 2 dimenstional vector because we set dout to 2
#
# W_query.to(device)
# W_key.to(device)
# W_value.to(device)
#
# Now the dot product with the input
#
query_2 = x_2 @ W_query
key_2   = x_2 @ W_key
value_2 = x_2 @ W_value
#
# query_2.to(device)
# key_2.to(device)
# value_2.to(device)
#
print("Query 2: ", query_2)
print("Key 2:   ", key_2)
print("Value 2: ", value_2)
#
print("\n")
#

keys = inputs @ W_key 
values = inputs @ W_value
# keys.to(device)
# values.to(device)
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys_2 = keys[1]

attn_score_22 = query_2.dot(keys_2)
# attn_score_22.to(device)
# attn_score_22 = query_2 @ keys_2
print("Attention (dot) score 22:", attn_score_22)

# Generalizing across all inputs
attn_scores_2 = query_2 @ keys.T
# attn_scores_2.to(device)
print("Attention \\@ Scores 2: ", attn_scores_2)
# Check the second element is same as previously calculated attention score
#
# We compute the attention weights by scaling the attention scores and using the softmax function. 
# However, now we scale the attention scores by dividing them by the square root of the embedding 
# dimension of the keys

attn_weights_2 = torch.softmax(attn_scores_2 / keys.shape[-1]**0.5, dim=-1)
# attn_weights_2.to(device)
print("attn_weights_2: ", attn_weights_2)

# The reason for the normalization by square root of embedding dimension size is to improve the training performance by avoiding small gradients. 
# For instance, when scaling up the embedding dimension, which is typically > 1,000 for GPT-like LLMs, large dot products can result in 
# very small gradients during backpropagation (due to the softmax function applied to them). 
# As dot products increase, the softmax function behaves more like a step function, resulting in gradients nearing zero. 
# These small gradients can drastically slow down learning or cause training to stagnate.
#
# The scaling by the square root of the embedding dimension is the reason why this self-attention mechanism is also called scaled-dot product attention.
# Similar to when we computed the context vector as a weighted sum over the input vectors 
# we now compute the context vector as a weighted sum over the value vectors. 
# Here, the attention weights serve as a weighting factor that weighs the respective importance of each value vector
context_vec_2 = attn_weights_2 @ values
# context_vec_2.to(device)
print("context_vec_2: ", context_vec_2)


In [ ]:
from src.chapter03.SelfAttention_v2 import SelfAttention_v2

torch.manual_seed(789)
self_attn_v2 = SelfAttention_v2(d_in, d_out)
#print("Context vectors from SelfAtten_v2: \n", self_attn_v2(inputs))

# Note since the input contains 6 embedding vectors, the output also has 6 rows of context vectors


# Causal Attention 
# First we apply softmax to the attention scores then mask with 0 above the diagonal and then normalize the rows to 1
#
queries = self_attn_v2.W_query(inputs)
# queries.to(device)

keys = self_attn_v2.W_key(inputs)
# keys.to(device)

attn_scores = queries @ keys.T
# attn_scores.to(device)

attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
# attn_weights.to(device)
#print("Attention Wrights: \n",attn_weights)

# Now mask the values above diagonal as 0 using the tril() function
#
context_length = attn_scores.shape[0]
#
mask_simple = torch.tril(torch.ones(context_length, context_length))
#print("Mask Simple: \n", mask_simple)
#
# Now simply multiply them to prevent the look ahead 
#
masked_attention_weights = attn_weights * mask_simple
# masked_attention_weights.to(device)
#print("Masked attention weights: \n", masked_attention_weights)

#
# Now re-normalize to make sure rows add up to 1. To do this we divide each element by sum of each row
#
row_sums = masked_attention_weights.sum(dim=-1, keepdim=True)
#print("row_sums: \n", row_sums)
masked_simple_norm = masked_attention_weights / row_sums
print("Masked & re-normalized weights: \n", masked_simple_norm)

In [ ]:
# A more efficient way to obtain masked attention weights is to mask the attention scores with 
# negative infinity before applying softmax function. (e^negative infinity -> 0)
# We can implement this masking by replacing values above the diagonal with 1 and then replacing them 
# with negative infinity
#
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
# mask.to(device)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
# masked.to(device)
print("Masked attention weights: \n", masked)
#
# Now apply the softmax function 
#
attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=-1)
print("Softmax'd Attention weights: \n", attn_weights)

# Now we can use these modified attention weights to calculate the context vector
#
context_vec = attn_weights @ values
print("context_vec: \n", context_vec)

In [ ]:
# Masking additional weights with dropout
# Drop out in the attention mechanism is applied at 2 specific times: 
# 1. After calculating the attention weights
# 2. After applying the attention weights to value vectors
# Here we will apply the dropout mask after computing the attention weights
#
# Lets use a dropout rate of 50% meaning half the attention weights will be masked out. 
# Normally it's a much lower rate like 0.1 or 0.2
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(dropout(example))
#
# Since we are applying 50% dropout, to compensate for reduction in active elements
# we are going to scale up the values of remaining elements by a factor of 1/0.5 = 2
# This scaling is crucial to maintain the balance of the attention weights
#
torch.manual_seed(123)
print("Dropped out attention weights: \n", dropout(attn_weights))

In [ ]:
from src.chapter03.CausalAttention import CausalAttention

# Let’s ensure that the code can handle batches consisting of more than one input so that 
# the CausalAttention class supports the batch outputs produced by the data loader
# To simulate batch input lets duplicate the input text
#
batch = torch.stack((inputs, inputs), dim=0)
# batch.to(device)
print("batch: \n", batch.shape)
# print(batch)
#
# We can now use the CausalAttention class as follows
#
torch.manual_seed(123)
context_layer = batch.shape[1]
causal_attn = CausalAttention(d_in, d_out, context_length, 0.0, False)
context_vecs = causal_attn(batch)
print("context_vecs: \n", context_vecs.shape)

In [ ]:
from src.chapter03.MultiHeadAttentionWrapper import MultiHeadAttentionWrapper

# Multi Head Attention
# Now if we use the MultiHeadAttentionWrapper class with two attention heads, and CausalAttention 
# output dimension d_out = 2, we get a 4 dimensional context vector (d_out * num_heads = 4).
#
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2
multi_head_attn = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout=0.0, num_heads=2, qkv_bias=False)
context_vecs = multi_head_attn(batch)
print("context_vecs: \n", context_vecs)
print("context_vecs.shape: \n", context_vecs.shape)


In [ ]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])
# print(a.transpose(2, 3))
a.to(device)

first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

print(f"Batched: \n{a @ a.transpose(2, 3)}")

In [ ]:
from src.chapter03.MultiHeadAttention import MultiHeadAttention

torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
print("Batch Shape: \n", batch.shape)
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("Context vector shape: \n", context_vecs.shape)



---
## Chapter 4: Implementing GPT from Scratch to generate text

---

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False,
    "model_name": "GPTModel",
    "lr": 5e-4,
    "weight_decay": 0.1
}

## Here is the proposed architecture and order of implementation
![image](../data/4-3.png)


In [ ]:
from src.chapter04.DummyGPTModel import DummyGPTModel
import tiktoken
#
torch.manual_seed(123)
#
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.clear()
batch.append(torch.tensor(tokenizer.encode(txt1)).to(device))
batch.append(torch.tensor(tokenizer.encode(txt2)).to(device))
batch = torch.stack(batch, dim=0).to(device)
batch.to(device)
print("Input Batch: \n", batch)
print("Input batch shape: \n", batch.shape)
#
# Next, we initialize a new 124-million-parameter DummyGPTModel instance 
# and feed it the tokenized batch
#
model = DummyGPTModel(GPT_CONFIG_124M)
model.to(device)
logits = model(batch)
print("Output shape: \n", logits.shape)
#print(logits)
#                      
#

In [ ]:
import torch
from torch import nn

# Let’s now implement LAYER Normalization to improve the stability and efficiency of the training.
# The main idea behind LAYER Normalization is to adjust the activations (outputs) of a deep
# neural network layer to have a mean of 0 and a variance of 1
#
# This adjustment speeds up the convergence.
#
torch.manual_seed(123)
torch.set_printoptions(sci_mode=False)
#
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5,6), nn.ReLU())
out = layer(batch_example)
print("Layer: \n",out)
#
# The NN Layer contains the non-linear activation ReLU which 0's out the negative values
# 
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
print("Mean: ", mean)
print("Variance: ", var)
print("\n")
#
# Next, let’s apply layer normalization to the layer outputs we obtained earlier. 
# The operation consists of subtracting the mean and dividing by the square root 
# of the variance (also known as the standard deviation):
#
eps = 1e-5
out_norm = (out - mean) / torch.sqrt(var)
# Dim = -1 indicates statistics along the last dimention
mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
torch.set_printoptions(sci_mode=True)
print("Normalized Layer Outputs: \n", out_norm)
print("Mean: \n", mean)
print("Variance: \n", var)
print("-------------------------\n")

torch.set_printoptions(sci_mode=False)
print("Mean:\n", mean)
print("Variance:\n", var)


In [ ]:
from src.chapter04.LayerNorm import LayerNorm

# Previously we used unbiased = False in our variance calculation. This doesn't 
# apply Bessel's correction where divisor is n-1 instead of n. But this is 
# compatible with GPT-2

ln = LayerNorm(emb_dim = 5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print("Mean: \n", mean)
print("Variance: \n", var)


In [ ]:
# Let us see how the GELU (Gaussian Error Linear Unit) stacks up against 
# # RELU (REctified Linear Unit)
from src.chapter04.GELU import GELU
import matplotlib.pyplot as plt
gelu, relu = GELU(), nn.ReLU()

x = torch.linspace(-3, 3, 100)
y_gelu, y_relu = gelu(x), relu(x)
plt.figure(figsize=(8, 3))
for i, (y, label) in enumerate(zip([y_gelu, y_relu], ["GELU", "ReLU"]), 1):
    plt.subplot(1, 2, i)
    plt.plot(x, y)
    plt.title(f"{label} activation function")
    plt.xlabel("x")
    plt.ylabel(f"{label}(x)")
    plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from src.chapter04.SimpleFeedForward import SimpleFeedForward

# As we can see the smoothness of the GELU can lead to better optimization properties during training
# as it allows more nuanced finer adjustments to models parameters. In contrast, RELU has a sharp corner
# that can make adjustments difficult for very deep networks.
#
# Next we look at implementing a feed forward network with GELU activations
# See SimpleFeedForward.py
#
sff = SimpleFeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768)
out = sff(x)
print(out.shape)

In [ ]:
from src.chapter04.ExampleDeepNeuralNetwork import ExampleDeepNeuralNetwork

# Next we implement Shortcut Connections
# Each layer will be initialized such that it accepts an example with three input 
# values and returns three output values.
torch.manual_seed(123)
layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
model_without_shortcut = ExampleDeepNeuralNetwork(layer_sizes, use_shortcut=False)
# model.to(device)

# Next lets print the gradients
def print_gradients(nnmodel, input_x):
    output = nnmodel(input_x)
    target = torch.tensor([[0.]])
    loss = nn.MSELoss()
    loss = loss(output, target)
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    for name, param in nnmodel.named_parameters():
        # print(name, " = ", param)
        if 'weight' in name:
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")

#
# Now Lets use this function to print the gradients calculated by loss.backward()
print_gradients(model_without_shortcut, sample_input)


In [ ]:
# As you can see above gradients become tiny aka Vanishing from Layer4 to Layer1
# Let’s now instantiate a model with skip connections and see how it compares:
torch.manual_seed(123)
layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
model_with_shortcut = ExampleDeepNeuralNetwork(layer_sizes, use_shortcut=True)
# model.to(device)
print_gradients(model_with_shortcut, sample_input)


In [ ]:
# Note here the gradient doesn't approach a vanishingly small value during backprop.
# In conclusion, shortcut connections are important for overcoming the limitations posed 
# by the vanishing gradient problem in deep neural networks.

#### Next, we’ll connect all the previously covered concepts (layer normalization, GELU activations, feed forward module, and shortcut connections) in a transformer  block, which is the final building block we need to code the GPT architecture.

![image](../data/transformer_wiring.png)

In [ ]:
# See TransformerBlock.py for the basic sequence and feedforward details
from src.chapter04.TransformerBlock import TransformerBlock

torch.manual_seed(123)
x = torch.rand(2, 4, 768)
tr_block = TransformerBlock(GPT_CONFIG_124M)
out = tr_block(x)
#
print("Input Shape:  ", x.shape)
print("Output Shape: ", out.shape)


In [ ]:
from src.chapter04.GPTModel import GPTModel 

# Let us wire up the actual GPT Model we wrote now

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
print("Input batch: ", batch)
out = model(batch)
print("Output shape: ", out.shape)
# print("Out: \n", out)


In [ ]:
# Note above the output tensor has the shape [2, 4, 50257], since we passed in two input texts (the two sentences) 
# with four tokens each. The last dimension, 50257, corresponds to the vocabulary size of the tokenizer.
#
# To capture the total number of Parameters for a model use numel parameter value
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

In [ ]:
# Weight Tying: The model reuses weights from the token embedding layer in its output layer
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)
# As we can see both shapes are same

In [ ]:
# The token embedding and output layers are very large due to the 50,257 rows in the tokenizer’s vocabulary. 
# If we remove the output layer parameter count from the total GPT-2 model count :
total_params_gptmodel = (
    total_params - sum(p.numel() for p in model.out_head.parameters())
)
print(f"Total number of trainable parameters considering weight tying: "
      f"{total_params_gptmodel:,}")

# Memory Requirement
total_size_bytes = total_params * 4
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

# Generating text 
#### We will now write code to generate text from the predicted tensors by the GPTModel


In [ ]:
def generate_text_simple(input_model, tokenids, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        token_cond = tokenids[:, -context_size:]
        with torch.no_grad():
            logits = input_model(token_cond)

        logits = logits[:, -1, :]
        probabs = torch.softmax(logits, dim=-1)
        next_token = torch.argmax(probabs, dim=-1, keepdim=True)
        tokenids = torch.cat((tokenids, next_token), dim=1)

        return tokenids
#    
#  Try it with a sample sentence  
#    
start_context = "Hello, I am "
encoded = tokenizer.encode(start_context)
print("encoded: ", encoded)
#
encoded_tensor = torch.tensor(encoded).to(device).unsqueeze(0)
print("encoded_tensor.shape: ", encoded_tensor.shape)
#
model.eval()
out = generate_text_simple(
    input_model=model,
    tokenids=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

In [ ]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

# Chapter 5: Pretraining on unlabeled data
#### Using GPT to generate text

In [ ]:
# How to calculate loss and relatively randomize next word prediction
#
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]]).to(device)   #  "I really like"
#
targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107, 588, 11311]]).to(device)  #  " really like chocolate"

with torch.no_grad():
    logits = model(inputs)
probas = torch.softmax(logits, dim=-1)
print(f"probas: {probas.shape}")

token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print(f"Token IDs: {token_ids}")

# Utility function
def text_to_token_ids(txt, tokenizr):
    encoded_txt = tokenizr.encode(txt, allowed_special={'<|endoftext|>'})
    encoded_tnsr = torch.tensor(encoded_txt).to(device).unsqueeze(0)
    return encoded_tnsr

# Utility function
def token_ids_to_text(tokenids, tokenizr):
    flat = tokenids.squeeze(0)
    return tokenizr.decode(flat.tolist())


print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

torch.set_printoptions(sci_mode=True)
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Softmax scores for Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Softmax scores for Text 2: ", target_probas_2)

# The goal of training an LLM is to maximize the likelihood of the correct token, 
# which involves increasing its probability relative to other tokens. 

# Loss of probabilities for the two batches are
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print("Log probabilities for both batches are: \n", log_probas)

# Next, we combine the log probabilities into a single score by computing 
# the average
avg_log_probas = torch.mean(log_probas)
print(f"Average log probability: {avg_log_probas}")

# However, in deep learning, the common practice isn’t to push the average log probability 
# up to 0 but rather to bring the negative average log probability down to 0. The negative 
# average log probability is simply the average log probability multiplied by –1
neg_avg_log_probabs = avg_log_probas * -1
print(f"Negative of average log probability: {neg_avg_log_probabs}")

# In deep learning, the term for turning this negative value, –10.7940, into 10.7940, 
# is known as the cross entropy loss.


#### NOTE: Our goal during training is to get the average log probability as close to 0 as possible by updating the model’s weights

In [ ]:
GPT_CONFIG_124M_2 = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12, 
    "drop_rate": 0.1,
    "qkv_bias": False,
    "model_name": "GPTModel",
    "lr": 5e-4,
    "weight_decay": 0.1
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(torch.device("mps"))
model.eval()



start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    input_model=model,
    tokenids=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


In [ ]:
print("Logits shape:", logits.shape) # batch size, num of tokens, vocab size
print("Targets shape:", targets.shape) # batch size, num of tokens

# For the cross_entropy loss function in PyTorch, we want to flatten these 
# tensors by combining them over the batch dimension:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()
print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

# Now we can call CE from torch to calculate the loss
celoss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(f"Cross Entropy loss: {celoss}")

# Perplexity measures how well the probability distribution predicted by the model 
# matches the actual distribution of the words in the dataset. Similar to the loss, 
# a lower perplexity means the model predictions are closer to the actual distribution.
perplexity = torch.exp(celoss)
print(f"Perplexity: {perplexity}")


In [ ]:
from src.chapter02.Dataloader import Dataloader
import torch.nn.functional as F
# To implement the data splitting and loading, we first define a train_ratio 
# to use 90% of the data for training and the remaining 10% as validation data 
# for model evaluation during training
torch.manual_seed(123)
# 
train_ratio = 0.9
split_idx = int(train_ratio * len(raw_text))
train_data = raw_text[:split_idx]
val_data = raw_text[split_idx:]
# 
train_loader = Dataloader(
    batch_size=2,
    max_length=GPT_CONFIG_124M_2["context_length"],
    stride=GPT_CONFIG_124M_2["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
).create_dataloader_v1(train_data)
# 
val_loader = Dataloader(
    batch_size=2,
    max_length=GPT_CONFIG_124M_2["context_length"],
    stride=GPT_CONFIG_124M_2["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
).create_dataloader_v1(val_data)
# 
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)
# 
print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)
#     
# Calculate per batch loss 
# Use CrossEntropy
# 
def calculate_batch_loss(input_batch, target_batch, model, device):
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)      
    batch_logits = model(input_batch)
    batch_loss = F.cross_entropy(
        batch_logits.flatten(0, 1), 
        target_batch.flatten()
    )
    # batch_loss = F.nll_loss(
    #     batch_logits.flatten(0, 1), 
    #     target_batch.flatten()
    # )
    return batch_loss

# 
# Calculate loss across batches
# 
def calculate_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for n, (input_batch, target_batch) in enumerate(data_loader):
        if n < num_batches:
            loss = calculate_batch_loss(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches
# 
model.to(device)
with torch.no_grad():
    train_loss = calculate_loss_loader(train_loader, model, device)
    val_loss = calculate_loss_loader(val_loader, model, device)
print("Default Training loss:", train_loss)
print("Default Validation loss:", val_loss)
# 
# Model Evaluation
#   Calculate both training and validation losses and return
#
def evaluate_model(eval_model, training_loader, validn_loader, eval_device, eval_iter):
    eval_model.eval()
    with torch.no_grad():
        training_loss = calculate_loss_loader(
            training_loader, eval_model, eval_device, num_batches=eval_iter
        )
        validn_loss = calculate_loss_loader(
            validn_loader, eval_model, eval_device, num_batches=eval_iter
        )
    eval_model.train()
    return training_loss, validn_loss

# A convenience function that we use to track whether the model improves during the training. 
# The generate_and_print_sample() function takes a text snippet (start_context) as input, 
# converts it into token IDs, and feeds it to the LLM to generate a text sample 
# using the generate_text_simple function
def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            input_model=model, 
            tokenids=encoded,
            max_new_tokens=50, 
            context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))
    model.train()


# 
# Now we implement the model training flow 
#
def train_model_simple(model, train_loader, val_loader,
                       optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # 
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1
    # 
    model.train()
    model.to(device)
    # 
    for epoch in range(num_epochs):
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()
            
            input_batch = input_batch.to(device)
            target_batch = target_batch.to(device)
            
            loss = calculate_batch_loss(input_batch, target_batch, model, device)
            loss.backward() # Back propagation
            # Optimizer step function with no closure supplied is below.
            # A closure should clear the gradients, compute the loss, and return it.
            # For example if it was a Conjugate Grad. optimization method 
            # We had to pass a closure. A closure is nothing but a function
            # for input, target in dataset:
            #    def closure():
            #        optimizer.zero_grad()
            #        output = model(input)
            #        loss = loss_fn(output, target)
            #        loss.backward()
            #        return loss
            #    optimizer.step(closure) 
            # 
            optimizer.step() 
            
            tokens_seen += input_batch.numel()
            global_step += 1

            if global_step % eval_freq == 0:
                trn_loss, validn_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter
                )
                train_losses.append(trn_loss)
                val_losses.append(validn_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {trn_loss:.3f}, "
                      f"Val loss {validn_loss:.3f}"
                )
        # End inner for
        generate_and_print_sample(model, tokenizer, device, start_context) # After each epoch
    # End outer for
    return train_losses, val_losses, track_tokens_seen




In [ ]:
#
# Training Loop
#
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M_2)
model.to(device)
optimizer = torch.optim.AdamW(
     model.parameters(),
    lr=0.0004, 
    weight_decay=0.1
)
num_epochs = 10
# 
train_losses, val_losses, tokens_seen = (
    train_model_simple(model, train_loader, val_loader, optimizer, device, 
                       num_epochs=num_epochs, eval_freq=5, eval_iter=5, 
                       start_context="Every effort moves you", tokenizer=tokenizer
                       )
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(figsize=(6, 4))
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(
        epochs_seen, val_losses, linestyle="-.", label="Validation loss"
    )
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax2 = ax1.twiny()
    ax2.plot(tokens_seen, train_losses, alpha=0)
    ax2.set_xlabel("Tokens seen")
    fig.tight_layout()
    plt.show()

epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    input_model=model,
    tokenids=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

# Now lets look at Temperature Scaling
vocab = { 
    "closer": 0,
    "every": 1, 
    "effort": 2, 
    "forward": 3,
    "inches": 4,
    "moves": 5, 
    "pizza": 6,
    "toward": 7,
    "you": 8,
} 
inverse_vocab = {v: k for k, v in vocab.items()}
# Assume the model generates the following logits 
next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
).to(device)

# We now generally do an argmax of the probabilities 
probas = torch.softmax(next_token_logits, dim=0).to(device)
next_token_id = torch.argmax(probas).item()
print(f"Next token (argmax): {inverse_vocab[next_token_id]}")
#
# But we can try replacing argmax with multinomial and get a sampling
#
torch.manual_seed(123) 
next_token_id = torch.multinomial(probas, num_samples=1).item()
print(f"Next token (multinomial): {inverse_vocab[next_token_id]}")
# 
# Let's see if multinomial can produce any other probabilites
def print_sampled_tokens(probabs):
    torch.manual_seed(123)
    # Multinomial Sampling of probabilities instead of max
    samples = [torch.multinomial(probabs, num_samples=1).item() for _ in range(1_000)]
    sampled_ids = torch.bincount(torch.tensor(samples).to(device))
    for cnt, freq in enumerate(sampled_ids):
        print(f"Sampled Freq. {freq} : {inverse_vocab[cnt]}")

print_sampled_tokens(probas)

In [ ]:
# We can further control the distribution by a technique called temperature scaling
# meaning dividing the logits with a >0 number
def softmax_with_temperature(logits, temperature, dim):
    scaled_logits = logits / temperature
    scaled_probs =  torch.softmax(scaled_logits, dim=dim)
    return scaled_probs

# Let's check that out
temperatures = [1, .01, 5]
next_token_logits = Tensor.cpu(next_token_logits)
scaled_probas = [softmax_with_temperature(next_token_logits, float(T), 0) for T in temperatures]
x = torch.arange(len(vocab))
bar_width = 0.5
fig, ax = plt.subplots(figsize=(6, 4))
for i, T in enumerate(temperatures):
    rects = ax.bar(x + i * bar_width, 
                   scaled_probas[i], 
                   bar_width, 
                   label=f'Temperature = {T}'
    )
ax.set_ylabel('Probability')
ax.set_xlabel('Words')
ax.set_xticks(x)
ax.set_xticklabels(vocab.keys(), rotation=90)
ax.legend()
plt.tight_layout()
plt.show()



## Top K sampling

In [ ]:
# Previously we implemented a probabilistic sampling approach coupled with 
# temperature scaling to increase the diversity of the outputs.  This method 
# allows for the exploring of less likely but potentially more interesting and 
# creative paths in the generation process.
#
# Top-k sampling, when combined with probabilistic sampling and temperature 
# scaling, can improve the text generation results.
#
# Here we can restrict the sampled tokens to the top-k most likely tokens 
# and exclude all other tokens from the selection process by masking their 
# probability scores
# 
top_k = 3
torch.set_printoptions(sci_mode=False)
# 
top_logits, top_pos = torch.topk(next_token_logits, top_k)
print("Top Logits: ", top_logits)
print("Top Positions: ", top_pos)
print(f"Next token logits: {next_token_logits}")

# Pytorch WHERE function to set the logit values of tokens that are below the lowest 
# logit value within our top-three selection to negative infinity (-inf)
#
new_logits = torch.where(
    condition=next_token_logits < top_logits[-1],
    input=torch.tensor(float('-inf')),
    other=next_token_logits
)
print("New Logits: ", new_logits)

# Now apply the softmax
topk_probas = torch.softmax(new_logits, dim=0)
print(f"top k probabilities: {topk_probas}")
# 
# We can now apply the temperature scaling and multinomial function for probabilistic 
# sampling to select the next token among these three non-zero probability scores to 
# GENERATE THE NEXT TOKEN with more diversity.
# 
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):
    # print("Entering generate()..")
    # print(idx.shape)
    for i in range(max_new_tokens):
        # print(f"idx: [{i}]: {idx}")
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        #     
        logits = logits[:, -1, :] # ([1, 50257])
        # print(logits.shape)
        if top_k is not None:
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1] # Less than the lowest value of top k
            # Now mark the minvals with -inf, so softmax becomes 0
            logits = torch.where(
                logits < min_val,
                torch.tensor(float('-inf')).to(logits.device),
                logits
            )
        if temperature > 0.0:
            probs = softmax_with_temperature(logits, temperature, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        else: 
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)

        if idx_next == eos_id:
            break

        # Next word
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M_2["context_length"],
    top_k=25,
    temperature=1.4
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))



In [ ]:
# torch.save(model.state_dict(), f"/Users/amlanchatterjee/Documents/ws/python/PycharmProjects/SimpleLLMProject/models/{GPT_CONFIG_124M_2['model_name']}.pth")
MODEL_PATH = f"/Users/amlanchatterjee/Documents/ws/python/PycharmProjects/SimpleLLMProject/models/{GPT_CONFIG_124M_2['model_name']}.pth"
# 
try:
    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        }, 
        MODEL_PATH
    )
    print(f"Model saved at {MODEL_PATH}\n")
except Exception as e:
    print(f"Encountered exception : {e}")
    
# 

In [ ]:
# Load the model back
checkpoint = torch.load(MODEL_PATH, map_location=device,weights_only=True)
loaded_model = GPTModel(GPT_CONFIG_124M_2).to(device)
try:
    loaded_model.load_state_dict(checkpoint["model_state_dict"])
    optimizer = torch.optim.AdamW(loaded_model.parameters(), 
                                 lr=GPT_CONFIG_124M_2["lr"], 
                                 weight_decay=GPT_CONFIG_124M_2["weight_decay"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    print(f"Model and Optimizer successfully loaded from \n{MODEL_PATH}\n")
except Exception as e:
    print(f"Encountered exception : {e}")
    
loaded_model.train()
print("Model set to train mode")

In [ ]:
# Try training the model a couple of times more
# epochs = 2
# train_ratio = 0.9
# # 
# print(f"Model is on {next(model.parameters()).device}")  
# train_losses, val_losses, tokens_seen = (
#     train_model_simple(
#         loaded_model, 
#         train_loader, 
#         val_loader, 
#         optimizer, 
#         device, 
#         num_epochs=epochs, 
#         eval_freq=5, 
#         eval_iter=5, 
#         start_context="Every effort moves you", 
#         tokenizer=tokenizer
#     )
# )
# epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
# plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)

### OpenAI also shares the weights of larger models: 355M, 774M, and 1558M 
![image](../data/model_arch_stack.png)



In [ ]:
# Load the downloaded GPT Data
import urllib.request
from src.chapter05.gpt_download import download_and_load_gpt2

url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch05/01_main-chapter-code/gpt_download.py"
)
filename = url.split('/')[-1]
urllib.request.urlretrieve(url, filename)

settings, gpt_params = download_and_load_gpt2(model_size="124M", models_dir="../data/gpt2")
print(f"\nParams: {gpt_params.keys()}")
print(f"Settings: {settings}")
print(f"Token embedding layer weight tensor dimensions: {gpt_params["wte"].shape}")    


In [ ]:

# After loading the GPT-2 model weights into Python, we still need to transfer 
# them from the settings and params dictionaries into our GPTModel instance. 
# First, we create a dictionary that lists the differences between the 
# different GPT model sizes
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
model_name="gpt2-small (124M)"
# 
NEW_GPT_CONFIG = GPT_CONFIG_124M.copy()
NEW_GPT_CONFIG.update({"model_name": model_name})
# Update the value ex. {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
NEW_GPT_CONFIG.update(model_configs[model_name]) 

NEW_GPT_CONFIG.update({"context_length": 1024})
NEW_GPT_CONFIG.update({"qkv_bias": True})
# 
print(f"{model_name}: {model_configs[model_name]}")
print("NEW_GPT_CONFIG:\n"+"".join(f"\t{k}: {v}\n" for k, v in sorted(NEW_GPT_CONFIG.items())))
# 
newgpt = GPTModel(NEW_GPT_CONFIG).to(device)
newgpt.eval()
# 
# Before we assign the loaded openai weights into the model, we will first define 
# a small assign utility function that checks whether two tensors or arrays 
# (left and right) have the same dimensions or shape and returns the right tensor 
# as trainable PyTorch parameters
def assign(left: Tensor, right: Tensor) -> Tensor:
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch Left shape: {left.shape} Right shape: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right).to(device))
# 
print(f"Params: {gpt_params.keys()}")
# print(f"Params: {gpt_params}")
print(f"GPT Parameter Blocks Count: {len(gpt_params["blocks"])}")

In [ ]:
#
# Load OpenAI Weights into our GPTModel code
#
import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight, 
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias, 
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].sff.layers[0].weight = assign(
            gpt.trf_blocks[b].sff.layers[0].weight, 
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].sff.layers[0].bias = assign(
            gpt.trf_blocks[b].sff.layers[0].bias, 
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].sff.layers[2].weight = assign(
            gpt.trf_blocks[b].sff.layers[2].weight, 
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].sff.layers[2].bias = assign(
            gpt.trf_blocks[b].sff.layers[2].bias, 
            params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale, 
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift, 
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale, 
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift, 
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])
    

In [ ]:
# Now lets try to load the weights and see
load_weights_into_gpt(newgpt, gpt_params)
newgpt.to(device)
print("Loaded weights into GPTModel")

In [ ]:
# Now lets generate
torch.manual_seed(123)
token_ids = generate(
    model=newgpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=NEW_GPT_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))